In [1]:
import findspark
# TODO: your path will likely not have 'matthew' in it. Change it to reflect your path.
findspark.init('/home/chiziheng/spark-3.1.1-bin-hadoop2.7')

In [2]:
import os
import sys

from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, current_timestamp
from pyspark.sql.functions import split
from pyspark.sql.functions import window
from pyspark.sql.types import StructType, TimestampType

In [3]:
spark = SparkSession.builder.appName("test_stream_from_csv").getOrCreate()

In [4]:
userSchema = StructType()\
              .add("_c0", "integer")\
              .add("DateTime", TimestampType())\
              .add("Door", "integer")\
              .add("MotorTemp", "integer")\
              .add("MotorCurrent","integer")\
              .add("OpenCloseStroke","integer")\
              .add("MotorVoltage","integer")\
              .add("SigOpenCommand","integer")\
              .add("SigCloseCommand","integer")

In [5]:
lines = spark.readStream.format("socket").option("host", "localhost").option("port", 12344).load()

In [6]:
query = lines.writeStream.outputMode("append").format("console").option("truncate", False).start()

In [7]:
query.stop()

In [ ]:
# Create query stream with memory sink
queryStream = (df_count
 .writeStream
 .format("memory")
 .queryName("wiki_changes")
 .outputMode("update")
 .start())
In [5]:
from time import sleep
from IPython.display import clear_output
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

matplotlib.rc('font', family='DejaVu Sans')
sns.set(style="whitegrid")


try:
    i=1
    while True:
        # Clear output
        clear_output(wait=True)
        print("**********************")
        print("General Info")
        print("**********************")
        print("Run:{}".format(i))
        if (len(queryStream.recentProgress) > 0):
            print("Stream timestamp:{}".format(queryStream.lastProgress["timestamp"]))
            print("Watermark:{}".format(queryStream.lastProgress["eventTime"]["watermark"]))
            print("Total Rows:{}".format(queryStream.lastProgress["stateOperators"][0]["numRowsTotal"]))
            print("Updated Rows:{}".format(queryStream.lastProgress["stateOperators"][0]["numRowsUpdated"]))
            print("Memory used MB:{}".format((queryStream.lastProgress["stateOperators"][0]["memoryUsedBytes"]) * 0.000001))
            
        df = spark.sql(
                """
                    select
                        window.start
                        ,window.end
                        ,user
                        ,sum(count) count
                    from
                        wiki_changes
                    where
                        window.start = (select max(window.start) from wiki_changes)
                    group by
                        window.start
                        ,window.end
                        ,user
                    order by
                        4 desc
                    limit 10
                """
        ).toPandas()

        # Plot the total crashes
        sns.set_color_codes("muted")

        # Initialize the matplotlib figure
        plt.figure(figsize=(8,6))

        print("**********************")
        print("Graph - Top 10 users")
        print("**********************")
        try:
            # Barplot
            sns.barplot(x="count", y="user", data=df)

            # Show barplot
            plt.show()
        except ValueError:
            # If Dataframe is empty, pass
            pass

        print("**********************")
        print("Table - Top 10 users")
        print("**********************")
        display(df)
        
        print("**********************")
        print("Table - Count by aggregation window")
        print("**********************")
        df1 = spark.sql(
                """
                    select
                        window.start
                        ,window.end
                        ,sum(count) qty_lines
                        ,count(distinct user) qty_users
                    from
                        wiki_changes
                    group by
                        window.start
                        ,window.end
                    order by
                        window.start desc
                """
        ).toPandas()
        
        display(df1)
        
        sleep(10)
        i=i+1
except KeyboardInterrupt:
    print("process interrupted.")

In [9]:
def stop_stream_query(query, wait_time):
    """Stop a running streaming query"""
    while query.isActive:
        msg = query.status['message']
        data_avail = query.status['isDataAvailable']
        trigger_active = query.status['isTriggerActive']
        if not data_avail and not trigger_active and msg != "Initializing sources":
            print('Stopping query...')
            query.stop()
        time.sleep(0.5)

    # Okay wait for the stop to happen
    print('Awaiting termination...')
    query.awaitTermination(wait_time)

+---------+-----+
|timestamp|value|
+---------+-----+
+---------+-----+

+-----+
|value|
+-----+
+-----+



In [8]:
activity=spark.readStream \
    .format('rate')\
    .option('rowsPerSecond',3)\
    .csv(path='/home/chiziheng/Downloads/pyspark-streaming/5_structured/test_stream_csv', schema=userSchema)

In [9]:
query = activity\
          .writeStream\
          .trigger(processingTime='3 seconds')\
          .format("console")\
          .start()

In [17]:
%pwd

'/home/chiziheng/Downloads/pyspark-streaming/5_structured'

In [ ]:
# df = spark.read.load("70N.csv", format="csv", sep=",", inferSchema="true", header="true")
# df.createOrReplaceTempView('70N_table')
# df.printSchema()
# spark.sql("select DateTime, MotorCurrent from 70N_table").show(50)